# Candidate Nightly Epochs

Author: Melissa Graham

This notebook uses as input the data files of "probably-real" candidates that were generated by `source_detection_summary.ipynb` and are stored in `source_detection_summary_files/`.

Keep in mind that only candidates with >10 objects (detections) and a mean R/B score > 0.4 were written to these files.

All of the objects (detections) for a given night in a given filter are combined for all candidates, with the result of one photometric point per filter per night.

The results are written to `candidate_nightly_epochs_files/`:
 * `candidate_lightcurves.dat`
   * all photometric points for a given night are combined
   * file includes date, filter, mean magnitude, error, and mean R/B
   * includes limiting magnitude estimate for epochs of non-detection
 * `candidate_lightcurves_parameters.dat`
   * four parameters: timespan, minimum brightness, amplitude, and number of nights detected (epochs detected)
   * four measures per parameter: for any filter, and for each of g, r, i

## Set up 

In [ ]:
# import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

# from copy import deepcopy
import time

## Read the data files

All are in `source_detection_summary_files/`.

#### all exposures

In [ ]:
fnm = 'source_detection_summary_files/exposures.dat'
exp_field  = np.loadtxt( fnm, dtype='str',   usecols=(0) )
exp_cal    = np.loadtxt( fnm, dtype='str',   usecols=(1) )
exp_fbs    = np.loadtxt( fnm, dtype='str',   usecols=(2) )
exp_id     = np.loadtxt( fnm, dtype='int',   usecols=(3) )
exp_mjd    = np.loadtxt( fnm, dtype='float', usecols=(4) )
exp_fil    = np.loadtxt( fnm, dtype='str',   usecols=(5) )
exp_mlmg   = np.loadtxt( fnm, dtype='float', usecols=(6) )
exp_msee   = np.loadtxt( fnm, dtype='float', usecols=(7) )
exp_msks   = np.loadtxt( fnm, dtype='float', usecols=(8) )
exp_tnobj  = np.loadtxt( fnm, dtype='int',   usecols=(9) )
exp_tnobjg = np.loadtxt( fnm, dtype='int',   usecols=(10) )
del fnm

#### create arrays for the unique epochs from the exposures

In [ ]:
cx = np.where( exp_field == 'COSMOS' )[0]
ex = np.where( exp_field == 'ELAIS' )[0]

c_u_cal = np.unique( np.sort( exp_cal[cx] ) )
e_u_cal = np.unique( np.sort( exp_cal[ex] ) )
print( 'Number of unique epochs for COSMOS and ELAIS: ',len(c_u_cal),len(e_u_cal) )

del cx,ex

In [ ]:
c_u_cal_meanmjd = np.zeros( len(c_u_cal), dtype='float' )
e_u_cal_meanmjd = np.zeros( len(e_u_cal), dtype='float' )
c_u_cal_meanlmg = np.zeros( len(c_u_cal), dtype='float' )
e_u_cal_meanlmg = np.zeros( len(e_u_cal), dtype='float' )

for c in range(len(c_u_cal)):
    tx = np.where( (exp_field == 'COSMOS') & (exp_cal == c_u_cal[c]) )[0]
    c_u_cal_meanmjd[c] = np.mean( exp_mjd[tx] )
    c_u_cal_meanlmg[c] = np.mean( exp_mlmg[tx] )
    del tx

for e in range(len(e_u_cal)):
    tx = np.where( (exp_field == 'ELAIS') & (exp_cal == e_u_cal[e]) )[0]
    e_u_cal_meanmjd[e] = np.mean( exp_mjd[tx] )
    e_u_cal_meanlmg[e] = np.mean( exp_mlmg[tx] )
    del tx

#### all candidates

In [ ]:
fnm = 'source_detection_summary_files/candidates.dat'
cand_field    = np.loadtxt( fnm, dtype='str',   usecols=(0) )
cand_id       = np.loadtxt( fnm, dtype='str',   usecols=(1) )
cand_ra       = np.loadtxt( fnm, dtype='float', usecols=(2) )
cand_dec      = np.loadtxt( fnm, dtype='float', usecols=(3) )
cand_nobj     = np.loadtxt( fnm, dtype='int',   usecols=(4) )
cand_meanrb   = np.loadtxt( fnm, dtype='float', usecols=(5) )
cand_stdrb    = np.loadtxt( fnm, dtype='float', usecols=(6) )
cand_tspan    = np.loadtxt( fnm, dtype='float', usecols=(7) )
cand_lcamp    = np.loadtxt( fnm, dtype='float', usecols=(8) )
cand_nobj_g   = np.loadtxt( fnm, dtype='int',   usecols=(9) )
cand_meanrb_g = np.loadtxt( fnm, dtype='float', usecols=(10) )
cand_stdrb_g  = np.loadtxt( fnm, dtype='float', usecols=(11) )
cand_tspan_g  = np.loadtxt( fnm, dtype='float', usecols=(12) )
cand_lcamp_g  = np.loadtxt( fnm, dtype='float', usecols=(13) )
cand_nobj_r   = np.loadtxt( fnm, dtype='int',   usecols=(14) )
cand_meanrb_r = np.loadtxt( fnm, dtype='float', usecols=(15) )
cand_stdrb_r  = np.loadtxt( fnm, dtype='float', usecols=(16) )
cand_tspan_r  = np.loadtxt( fnm, dtype='float', usecols=(17) )
cand_lcamp_r  = np.loadtxt( fnm, dtype='float', usecols=(18) )
cand_nobj_i   = np.loadtxt( fnm, dtype='int',   usecols=(19) )
cand_meanrb_i = np.loadtxt( fnm, dtype='float', usecols=(20) )
cand_stdrb_i  = np.loadtxt( fnm, dtype='float', usecols=(21) )
cand_tspan_i  = np.loadtxt( fnm, dtype='float', usecols=(22) )
cand_lcamp_i  = np.loadtxt( fnm, dtype='float', usecols=(23) )
del fnm

print('Total number of candidates: ',len(cand_id))

#### candidates' objects data ("detections")

In [ ]:
fnm = 'source_detection_summary_files/candidate_objects.dat'
cob_field   = np.loadtxt( fnm, dtype='str',   usecols=(0) )
cob_candid  = np.loadtxt( fnm, dtype='str',   usecols=(1) )
cob_objid   = np.loadtxt( fnm, dtype='str',   usecols=(2) )
cob_objra   = np.loadtxt( fnm, dtype='float', usecols=(3) )
cob_objdec  = np.loadtxt( fnm, dtype='float', usecols=(4) )
cob_objmjd  = np.loadtxt( fnm, dtype='float', usecols=(5) )
cob_objfil  = np.loadtxt( fnm, dtype='str',   usecols=(6) )
cob_objmag  = np.loadtxt( fnm, dtype='float', usecols=(7) )
cob_objmage = np.loadtxt( fnm, dtype='float', usecols=(8) )
cob_objrb   = np.loadtxt( fnm, dtype='float', usecols=(9) )
cob_expid   = np.loadtxt( fnm, dtype='float', usecols=(10) )
del fnm

## Combine the photometry per night, per filter

Include non-detections and limiting magnitudes, from the exposures arrays.

In [ ]:
t1 = time.time()

temp_ne_field = []
temp_ne_candid = []
temp_ne_cal = []
temp_ne_mjd = []
temp_ne_fil = []
temp_ne_nobj = []
temp_ne_mag = []
temp_ne_mage = []
temp_ne_mrb = []
temp_ne_lmag = []


### do each candidate in turn
for c in range(len(cand_field)):
    tally_ne = 0
    
    if (c == 10) | (c == 100) | (c == 1000):
        t2 = time.time()
        print(c, ((t2-t1)/float(c))*(float(len(cand_field)-c)),' remain')
    
    ### the unique epochs for this field
    if cand_field[c] == 'COSMOS':
        f_u_cal = c_u_cal
        f_u_cal_meanmjd = c_u_cal_meanmjd
        f_u_cal_meanlmg = c_u_cal_meanlmg
    elif cand_field[c] == 'ELAIS':
        f_u_cal = e_u_cal
        f_u_cal_meanmjd = e_u_cal_meanmjd
        f_u_cal_meanlmg = e_u_cal_meanlmg
        
    ### all candidate-object rows for this candidate
    cx = np.where(cob_candid == cand_id[c])[0]
    
    ### consider every epoch (night) for this candidate's field in turn
    for d,mjd in enumerate(f_u_cal_meanmjd):
        
        ### consider each filter in turn
        for f,fil in enumerate(['g','r','i']):
            fx = np.where( (np.abs(cob_objmjd[cx] - mjd) < 0.4) & \
                           (cob_objfil[cx] == fil) )[0]
            
            ### two or more detections in this filter in this night
            if len(fx) >= 2:
                temp_ne_field.append( cand_field[c] )
                temp_ne_candid.append( cand_id[c] )
                temp_ne_cal.append( f_u_cal[d] )
                temp_ne_mjd.append( np.mean(cob_objmjd[cx[fx]]) )
                temp_ne_fil.append( fil )
                temp_ne_nobj.append( len(fx) )
                temp_ne_mag.append( np.mean(cob_objmag[cx[fx]]) )
                temp_ne_mage.append( np.sqrt(np.mean(cob_objmage[cx[fx]])**2 + \
                                                np.std(cob_objmag[cx[fx]])**2) )
                temp_ne_mrb.append( np.mean(cob_objrb[cx[fx]]) )
                temp_ne_lmag.append( f_u_cal_meanlmg[d] )
                tally_ne += 1
            
            ### only one detection in this filter on this night
            elif len(fx) == 1:
                temp_ne_field.append( cand_field[c] )
                temp_ne_candid.append( cand_id[c] )
                temp_ne_cal.append( f_u_cal[d] )
                temp_ne_mjd.append( cob_objmjd[cx[fx[0]]] )
                temp_ne_fil.append( fil )
                temp_ne_nobj.append( 1 )
                temp_ne_mag.append( cob_objmag[cx[fx[0]]] )
                temp_ne_mage.append( cob_objmage[cx[fx[0]]] )
                temp_ne_mrb.append( cob_objrb[cx[fx[0]]] )
                temp_ne_lmag.append( f_u_cal_meanlmg[d] )
                tally_ne += 1
                
            ### no detections in this filter on this night
            elif len(fx) == 0:
                temp_ne_field.append( cand_field[c] )
                temp_ne_candid.append( cand_id[c] )
                temp_ne_cal.append( f_u_cal[d] )
                temp_ne_mjd.append( f_u_cal_meanmjd[d] )
                temp_ne_fil.append( fil )
                temp_ne_nobj.append( 0 )
                temp_ne_mag.append( float('NaN') )
                temp_ne_mage.append( float('NaN') )
                temp_ne_mrb.append( float('NaN') )
                temp_ne_lmag.append( f_u_cal_meanlmg[d] )
            del fx
    
    if tally_ne == 0:
        print('warning: ', cand_id[c], ' no nightly epochs with detections')
    del cx

ne_field  = np.asarray( temp_ne_field, dtype='str' )
ne_candid = np.asarray( temp_ne_candid, dtype='str' )
ne_nobj   = np.asarray( temp_ne_nobj, dtype='int' )
ne_mjd    = np.asarray( temp_ne_mjd, dtype='float' )
ne_cal    = np.asarray( temp_ne_cal, dtype='str' )
ne_fil    = np.asarray( temp_ne_fil, dtype='str' )
ne_mag    = np.asarray( temp_ne_mag, dtype='float' )
ne_mage   = np.asarray( temp_ne_mage, dtype='float' )
ne_mrb    = np.asarray( temp_ne_mrb, dtype='float' )
ne_lmag   = np.asarray( temp_ne_lmag, dtype='float' )

del temp_ne_field,temp_ne_candid,temp_ne_nobj,temp_ne_mjd,temp_ne_cal
del temp_ne_fil,temp_ne_mag,temp_ne_mage,temp_ne_mrb,temp_ne_lmag
del tally_ne

t2 = time.time()
print('elapsed: ',t2-t1)

In [ ]:
print(ne_field)
print(ne_candid)

### write nightly epoch candidate photometry to file

In [ ]:
fnm = 'candidate_nightly_epochs_files/candidate_lightcurves.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, candidate_nightly_epochs.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 field  \n')
fout.write('#  1 id -- candidate identifier \n')
fout.write('#  2 calendar date \n')
fout.write('#  3 MJD \n')
fout.write('#  4 filter \n')
fout.write('#  5 number of objects combined \n')
fout.write('#  6 magnitude (mean of objects combined) \n')
fout.write('#  7 magnitude error \n')
fout.write('#  8 real/bogus (mean of objects combined) \n')
fout.write('#  9 exposure limiting magnitude (nightly mean) \n')
fout.write('# \n')

for i in range(len(ne_candid)):
    fout.write('%-9s %-9s %-8s %12.6f '\
               '%1s %3i %6.3f %6.3f %6.4f %6.3f \n' % \
               (ne_field[i], ne_candid[i], ne_cal[i], ne_mjd[i], \
                ne_fil[i], ne_nobj[i], ne_mag[i], ne_mage[i], ne_mrb[i], ne_lmag[i]) )

fout.close()
print('Wrote to: ',fnm)
del fnm

## Create summary parameters for nightly-epoch lightcurves

In [ ]:
t1 = time.time()

### timespan (last date - first date of detection)
nelc_tspan = np.zeros( len(cand_field), dtype='float' )
nelc_tspan_g = np.zeros( len(cand_field), dtype='float' )
nelc_tspan_r = np.zeros( len(cand_field), dtype='float' )
nelc_tspan_i = np.zeros( len(cand_field), dtype='float' )

### minimum magnitude (brightest detection)
nelc_minmag = np.zeros( len(cand_field), dtype='float' )
nelc_minmag_g = np.zeros( len(cand_field), dtype='float' )
nelc_minmag_r = np.zeros( len(cand_field), dtype='float' )
nelc_minmag_i = np.zeros( len(cand_field), dtype='float' )

### amplitude (maximum - minimum detection)
nelc_lcamp = np.zeros( len(cand_field), dtype='float' )
nelc_lcamp_g = np.zeros( len(cand_field), dtype='float' )
nelc_lcamp_r = np.zeros( len(cand_field), dtype='float' )
nelc_lcamp_i = np.zeros( len(cand_field), dtype='float' )

### number of epochs with detections
nelc_nde = np.zeros( len(cand_field), dtype='int' )
nelc_nde_g = np.zeros( len(cand_field), dtype='int' )
nelc_nde_r = np.zeros( len(cand_field), dtype='int' )
nelc_nde_i = np.zeros( len(cand_field), dtype='int' )

### for every candidate
for c,candid in enumerate(cand_id):
    if (c == 10) | (c == 100) | (c == 1000):
        t2 = time.time()
        print(c, ((t2-t1)/float(c))*(float(len(cand_field)-c)),' remain')

    cx = np.where( (ne_candid == candid) & (ne_nobj >= 1) )[0]

    ### where the nobj >= 1, the mag should never be nan
    tx = np.where( np.isnan( ne_mag[cx] ) )[0]
    if len(tx) != 0:
        print('warning, ', candid, ' has a nobj >=1, mag=nan epoch')
    del tx

    temp = np.unique( ne_cal[cx] )
    nelc_nde[c] = len(temp)
    del temp
    nelc_tspan[c]  = np.max( ne_mjd[cx] ) - np.min( ne_mjd[cx] )
    nelc_minmag[c] = np.min( ne_mag[cx] )
    nelc_lcamp[c]  = np.max( ne_mag[cx] ) - np.min( ne_mag[cx] )
    del cx

    gx = np.where( (ne_candid == candid) & (ne_nobj >= 1) & (ne_fil == 'g') )[0]
    rx = np.where( (ne_candid == candid) & (ne_nobj >= 1) & (ne_fil == 'r') )[0]
    ix = np.where( (ne_candid == candid) & (ne_nobj >= 1) & (ne_fil == 'i') )[0]
    if len(gx) > 0:
        nelc_nde_g[c]    = len(gx)
        nelc_tspan_g[c]  = np.max( ne_mjd[gx] ) - np.min( ne_mjd[gx] )
        nelc_minmag_g[c] = np.min( ne_mag[gx] )
        nelc_lcamp_g[c]  = np.max( ne_mag[gx] ) - np.min( ne_mag[gx] )
    if len(rx) > 0:
        nelc_nde_r[c]    = len(rx)
        nelc_tspan_r[c]  = np.max( ne_mjd[rx] ) - np.min( ne_mjd[rx] )
        nelc_minmag_r[c] = np.min( ne_mag[rx] )
        nelc_lcamp_r[c]  = np.max( ne_mag[rx] ) - np.min( ne_mag[rx] )
    if len(ix) > 0:
        nelc_nde_i[c]    = len(ix)
        nelc_tspan_i[c]  = np.max( ne_mjd[ix] ) - np.min( ne_mjd[ix] )
        nelc_minmag_i[c] = np.min( ne_mag[ix] )
        nelc_lcamp_i[c]  = np.max( ne_mag[ix] ) - np.min( ne_mag[ix] )
    del gx,rx,ix

t2 = time.time()
print('elapsed: ',t2-t1)

### Write lightcurve parameters to file

In [ ]:
fnm = 'candidate_nightly_epochs_files/candidate_lightcurve_parameters.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, candidate_nightly_epochs.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 field  \n')
fout.write('#  1 id -- candidate identifier \n')
fout.write('#  2 timespan (days between first and last detection) \n')
fout.write('#  3 timespan in g \n')
fout.write('#  4 timespan in r \n')
fout.write('#  5 timespan in i \n')
fout.write('#  6 minimum magnitude (not necessarily the peak) \n')
fout.write('#  7 minimum magnitude g \n')
fout.write('#  8 minimum magnitude r \n')
fout.write('#  9 minimum magnitude i \n')
fout.write('# 10 amplitude (magnitudes between brightest and faintest detection) \n')
fout.write('# 11 amplitude g \n')
fout.write('# 12 amplitude r \n')
fout.write('# 13 amplitude i \n')
fout.write('# 14 number of epochs (number of unique nights detected) \n')
fout.write('# 15 number of epochs in g \n')
fout.write('# 16 number of epochs in r \n')
fout.write('# 17 number of epochs in i \n')
fout.write('# \n')

for i in range(len(cand_field)):
    fout.write('%-8s %-9s %7.2f %7.2f %7.2f %7.2f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %3i %3i %3i %3i \n' % \
               (cand_field[i], cand_id[i], \
                nelc_tspan[i], nelc_tspan_g[i], nelc_tspan_r[i], nelc_tspan_i[i], \
                nelc_minmag[i], nelc_minmag_g[i], nelc_minmag_r[i], nelc_minmag_i[i], \
                nelc_lcamp[i], nelc_lcamp_g[i], nelc_lcamp_r[i], nelc_lcamp_i[i], \
                nelc_nde[i], nelc_nde_g[i], nelc_nde_r[i], nelc_nde_i[i] ) )

fout.close()
print('Wrote to: ',fnm)
del fnm